In [6]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from google.colab import drive

# import tensorflow
import tensorflow as tf
from tensorflow import keras

# import keras
from keras.callbacks import EarlyStopping
from tensorflow.python.keras.models import load_model

# import sklearn
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error

# 1. Create Data Set

## 1-1. Google Drive connect and files load

In [4]:
drive.mount('/content/drive') # 코랩에서 구글 드라이브에 마운트

Mounted at /content/drive


In [64]:
drive_path = '/content/drive/MyDrive/PNUProject/AIProject/data'
file_list = os.listdir(drive_path)
file_list

['ma.csv', 'macd.csv', 'rsi.csv', 'bb.csv', 'historydata_v1.csv']

In [65]:
df_list = []
for i in range(0,len(file_list),1):
  path = drive_path + '/' + file_list[i]
  data = pd.read_csv(path, encoding='cp949')
  path = ''
  df_list.append(data)

In [74]:
df_list[1].tail()

,candel_date_time,MACD
19995,2023-02-20 11:00:00,-89300
19996,2023-02-20 12:00:00,-85000
19997,2023-02-20 13:00:00,-76800
19998,2023-02-20 14:00:00,-72400
19999,2023-02-20 15:00:00,-68100


## 1-2. Data Preprocessing

In [49]:
pdf.columns

Index(['candel_date_time', 'open', 'high', 'low', 'close', 'volume'], dtype='object')

In [75]:
# 1.보조지표 + 종가 데이터 셋 - cdf
cdf = pd.concat([df_list[4]['candel_date_time'],
                 df_list[4]['close'],
                 df_list[0]['MA_C15H'],
                 df_list[1]['MACD'],
                 df_list[2]['RSI'],
                 df_list[3]['BB+'],
                 df_list[3]['BBC'],
                 df_list[3]['BB-']], axis = 1)
# 2. 가격 데이터셋 - pdf
# Way1
df_list[4] = df_list[4][['candel_date_time','open', 'high', 'low', 'close', 'volume']]
pdf = df_list[4]
# Way2
# pdf = pd.concat([df_list[0]['candel_date_time'],
#                 df_list[0]['open'],
#                 df_list[0]['high'],
#                 df_list[0]['low'],
#                 df_list[0]['close'],
#                 df_list[0]['volume']], axis = 1)

In [96]:
# normalizing data
c_scaler = MinMaxScaler()
p_scaler = MinMaxScaler()
c_scale_cols = ['close', 'MA_C15H', 'MACD', 'RSI', 'BB+', 'BBC','BB-']
p_scale_cols = ['open', 'high', 'low', 'close', 'volume']
c_scaled = c_scaler.fit_transform(cdf[c_scale_cols])
p_scaled = p_scaler.fit_transform(pdf[p_scale_cols])
fcdf = pd.DataFrame(c_scaled, columns=c_scale_cols)
fpdf = pd.DataFrame(p_scaled, columns=p_scale_cols)

In [98]:
# Revert normalized data
rf_cdf = pd.DataFrame(c_scaler.inverse_transform(fcdf),columns=c_scale_cols)
rf_pdf = pd.DataFrame(p_scaler.inverse_transform(fpdf),columns=p_scale_cols)